# CPsyCounD.txt convert to CPsyCounD.json
* SFT dataset: CPsyCounD.json
* Data Format: llaMA-Factory

In [ ]:
import json
import re
import os

In [ ]:
visitor_pattern = re.compile(r'来访者：(.*?)\n(?=心理咨询师：)', re.S)
therapist_pattern = re.compile(r'心理咨询师：(.*?)(?=\n来访者：|\Z)', re.S)

def txt_to_sft_json(txt_content):
    visitor_dialogues = re.findall(visitor_pattern, txt_content)
    therapist_dialogues = re.findall(therapist_pattern, txt_content)

    sft_data = []
    history = []

    for i in range(len(visitor_dialogues)):
        if i < len(therapist_dialogues):
            history.append([visitor_dialogues[i], therapist_dialogues[i]])
            output_dialogue = therapist_dialogues[i]
        else:
            break

        sft_data.append({
            "instruction": visitor_dialogues[i],
            "input": "",
            "output": output_dialogue,
            "history": history[:-1]  
        })

    return sft_data

def save_as_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

def convert_folder_txt_to_json(input_folder_path, output_folder_path):
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for file_name in os.listdir(input_folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(input_folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as txt_file:
                content = txt_file.read()
            
            sft_data = txt_to_sft_json(content)
            # Check if client_dialogues and therapist_dialogues are empty
            if not sft_data:
                print(f"Error: No dialogues found in file {file_name}")
                continue
            
            json_file_path = os.path.join(output_folder_path, file_name.replace('.txt', '.json'))
            save_as_json(sft_data, json_file_path)



In [ ]:
folder_path = ''    # CPsyCounD.txt path
output_folder_path = ''  # CPsyCounD.json path

convert_folder_txt_to_json(folder_path, output_folder_path)

In [ ]:
with open('CPsyCounD.json', 'r', encoding='utf-8') as file:
    data = json.load(file)